In [95]:
import pandas as pd
import numpy as np
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/27_MCK_Health')
import matplotlib.pyplot as plt
from sklearn import cross_validation, metrics   #Additional scklearn functions

%matplotlib inline

df_train=pd.read_csv('01.RawData/train_ajEneEa.csv')
df_test=pd.read_csv('01.RawData/test_v2akXPA.csv')
df_subm=pd.read_csv('01.RawData/sample_submission_1.csv')

def get_signals(df1):
    df=df1.copy()
    gender_sigs=pd.get_dummies(df['gender']).add_prefix('sig_gender_')
    work_sigs=pd.get_dummies(df['work_type']).add_prefix('sig_work_')
    smoking_sigs=pd.get_dummies(df['smoking_status'].fillna('missing')).add_prefix('sig_smoke_')
    df=pd.concat([df,gender_sigs,work_sigs,smoking_sigs],axis=1)
    df['sig_id1']=df['id']
    df['sig_id2']=1.0*df.index/len(df)    
    for sig in ['age','hypertension','heart_disease','avg_glucose_level','bmi']:
        df['sig_%s'%sig]=df[sig].fillna(-9999)
    df['sig_married']=(df['ever_married']=='Yes').astype(np.int8)
    df['sig_residence']=(df['Residence_type']=='Urban').astype(np.int8)
    df['sig_bmi_missing']=df['bmi'].isnull().astype(np.int8)
    output_sigs=[x for x in df.columns if x[:3] in ['sig','str']]
    return df[output_sigs]

sigs_train=get_signals(df_train)
sigs_test=get_signals(df_test)

sigs_train['sample']=np.random.random(len(sigs_train))<0.8
idv=[x for x in sigs_train.columns if x[:3] in ['sig']]
X_ins=sigs_train[sigs_train['sample']==1][idv]
X_oos=sigs_train[sigs_train['sample']==0][idv]
Y_ins=sigs_train[sigs_train['sample']==1]['stroke']
Y_oos=sigs_train[sigs_train['sample']==0]['stroke']

In [103]:
#Train XGBoost for stroke probability
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

clf1 = XGBClassifier( learning_rate =0.01, n_estimators=500, max_depth=3, min_child_weight=1, gamma=0.0, subsample=0.7,
 colsample_bytree=0.45, reg_alpha=0.1, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
clf1.fit(X_ins, Y_ins,eval_metric='auc')


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.45,
       gamma=0.0, learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, nthread=4,
       objective='binary:logistic', reg_alpha=0.1, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7)

In [96]:
#Train Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions

clf2=RandomForestClassifier(n_estimators=50,max_depth=5,random_state=1234)
clf2.fit(X_ins, Y_ins)

print metrics.roc_auc_score(Y_ins, clf2.predict_proba(X_ins)[:,1])
print metrics.roc_auc_score(Y_oos, clf2.predict_proba(X_oos)[:,1])

0.882346903909
0.86630086271


In [97]:
#Train Logistic Regression
from sklearn.linear_model import LogisticRegression

clf3=LogisticRegression(C=0.1)
clf3.fit(X_ins, Y_ins)

print metrics.roc_auc_score(Y_ins, clf3.predict_proba(X_ins)[:,1])
print metrics.roc_auc_score(Y_oos, clf3.predict_proba(X_oos)[:,1])

0.834980925684
0.832767923227


In [101]:
from sklearn.neighbors import KNeighborsClassifier

clf4=KNeighborsClassifier(n_neighbors=1000)
clf4.fit(X_ins, Y_ins)
print metrics.roc_auc_score(Y_ins, clf4.predict_proba(X_ins)[:,1])
print metrics.roc_auc_score(Y_oos, clf4.predict_proba(X_oos)[:,1])

0.6190588966
0.555016165566


In [102]:
from  sklearn.ensemble import ExtraTreesClassifier
clf5=ExtraTreesClassifier(n_estimators=30, max_depth=4,random_state=1234)
clf5.fit(X_ins, Y_ins)
print metrics.roc_auc_score(Y_ins, clf5.predict_proba(X_ins)[:,1])
print metrics.roc_auc_score(Y_oos, clf5.predict_proba(X_oos)[:,1])

0.859716993104
0.853975269885


In [107]:
print metrics.roc_auc_score(Y_ins, clf1.predict_proba(X_ins)[:,1])
print metrics.roc_auc_score(Y_oos, clf1.predict_proba(X_oos)[:,1])

0.87766130865
0.87394730219


In [105]:
pred=[]
for clf in [clf1,clf2,clf5]:
    pred.append(clf.predict_proba(X_oos)[:,1])
    

In [106]:
from sklearn.linear_model import Lasso

reg=Lasso(positive=True,alpha=0,fit_intercept=False)
reg.fit(np.array(pred).T,Y_oos)
print reg.coef_

[ 1.14109162  0.          0.        ]


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  after removing the cwd from sys.path.


In [90]:
import pandas as pd
import numpy as np
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/27_MCK_Health')
import matplotlib.pyplot as plt
from sklearn import cross_validation, metrics   #Additional scklearn functions

%matplotlib inline

df_train=pd.read_csv('01.RawData/train_ajEneEa.csv')
df_test=pd.read_csv('01.RawData/test_v2akXPA.csv')
df_subm=pd.read_csv('01.RawData/sample_submission_1.csv')

def get_signals(df1):
    df=df1.copy()
    gender_sigs=pd.get_dummies(df['gender']).add_prefix('sig_gender_')
    work_sigs=pd.get_dummies(df['work_type']).add_prefix('sig_work_')
    smoking_sigs=pd.get_dummies(df['smoking_status'].fillna('missing')).add_prefix('sig_smoke_')
    df=pd.concat([df,gender_sigs,work_sigs,smoking_sigs],axis=1)
    for sig in ['age','hypertension','heart_disease','avg_glucose_level','bmi']:
        df['sig_%s'%sig]=df[sig].fillna(28.6)
    df['sig_married']=(df['ever_married']=='Yes').astype(np.int8)
    df['sig_residence']=(df['Residence_type']=='Urban').astype(np.int8)
    df['sig_bmi_missing']=df['bmi'].isnull().astype(np.int8)
    output_sigs=[x for x in df.columns if x[:3] in ['sig','str']]
    return df[output_sigs]

sigs_train=get_signals(df_train)
sigs_test=get_signals(df_test)

sigs_train['sample']=np.random.random(len(sigs_train))<0.8
idv=[x for x in sigs_train.columns if x[:3] in ['sig']]
X_ins=sigs_train[sigs_train['sample']==1][idv]
X_oos=sigs_train[sigs_train['sample']==0][idv]
Y_ins=sigs_train[sigs_train['sample']==1]['stroke']
Y_oos=sigs_train[sigs_train['sample']==0]['stroke']

X_ins.mean()

sig_gender_Female              0.588915
sig_gender_Male                0.410912
sig_gender_Other               0.000172
sig_work_Govt_job              0.125539
sig_work_Never_worked          0.004025
sig_work_Private               0.571063
sig_work_Self-employed         0.156184
sig_work_children              0.143190
sig_smoke_formerly smoked      0.172684
sig_smoke_missing              0.306273
sig_smoke_never smoked         0.370063
sig_smoke_smokes               0.150980
sig_age                       42.202634
sig_hypertension               0.093428
sig_heart_disease              0.046628
sig_avg_glucose_level        104.582566
sig_bmi                       28.605517
sig_married                    0.644426
sig_residence                  0.501754
sig_bmi_missing                0.033778
dtype: float64

In [93]:
X=(X_ins-X_ins.mean())/X_ins.std()
from sklearn.manifold import TSNE
X = TSNE(n_components=2).fit_transform(X)


MemoryError: 